In [1]:
def setProjectPath(projectPath):
    import os
    import sys
    root=os.path.abspath(projectPath)
    print(root)
    sys.path.append(root)
    os.environ["TORCH_HOME"]=r"E:\Data\torch-model"
    import warnings  
    warnings.filterwarnings("ignore", category=RuntimeWarning, module="mne")


In [2]:
projectPath=r'D:\project_meta\NNproject\NNI'
setProjectPath(projectPath)
import mne
datasetFolder=r'D:\dataset\driver_dataset\DROZY\DROZY'
outputFolder=r'D:\project_meta\NNproject\NNI\output'
from project.dataprocess.FolderTree import *
datasetDict=getDataPath(datasetFolder)
outputDict=getOutPath(outputFolder)
import pprint
pprint.pprint(datasetDict)
pprint.pprint(outputDict)

D:\project_meta\NNproject\NNI
{'annotation_auto_folder': 'D:\\dataset\\driver_dataset\\DROZY\\DROZY\\annotations-auto',
 'annotation_manual_folder': 'D:\\dataset\\driver_dataset\\DROZY\\DROZY\\annotations-manual',
 'kinect_file': 'D:\\dataset\\driver_dataset\\DROZY\\DROZY\\kinect-intrinsics.yaml',
 'kss_file': 'D:\\dataset\\driver_dataset\\DROZY\\DROZY\\KSS.txt',
 'psg_folder': 'D:\\dataset\\driver_dataset\\DROZY\\DROZY\\psg',
 'pvt_rt_folder': 'D:\\dataset\\driver_dataset\\DROZY\\DROZY\\pvt-rt',
 'videos_folder': 'D:\\dataset\\driver_dataset\\DROZY\\DROZY\\videos_i8'}
{'ECG_filted_folder': 'D:\\project_meta\\NNproject\\NNI\\output\\psg_filted\\ECG',
 'ECG_table_file': 'D:\\project_meta\\NNproject\\NNI\\output\\ECG_table.csv',
 'ECG_uniformfilted_folder': 'D:\\project_meta\\NNproject\\NNI\\output\\psg_filted\\ECG',
 'EEG_filted_folder': 'D:\\project_meta\\NNproject\\NNI\\output\\psg_filted\\EEG',
 'EEG_table_file': 'D:\\project_meta\\NNproject\\NNI\\output\\EEG_table.csv',
 'EEG_unifor

# 模型

In [3]:
from project.model.CNN1D import *
from torchvision.datasets import *
from torchvision.transforms import *
import torchvision
train_transforms = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
])
MNISTDataset=FashionMNIST(root=r'D:/dataset',transform=train_transforms)


In [4]:
print(MNISTDataset[0][0].shape)
flatten=nn.Flatten()
out=flatten(MNISTDataset[0][0])

torch.Size([1, 28, 28])


In [5]:
len(MNISTDataset.classes)

10

In [59]:
from typing import Any, Optional
import pytorch_lightning as pl
from pytorch_lightning.utilities.types import STEP_OUTPUT
from torchmetrics.functional import accuracy

class PLMLP(pl.LightningModule):
    def __init__(self, net,lossFuncthion,optimizer,lrScheduler,*args: Any, **kwargs: Any) -> None:
        super().__init__(*args, **kwargs)
        self.Flatten=nn.Flatten()
        self.Net=net
        self.LossFunction=nn.CrossEntropyLoss()
        self.Optimizer=optimizer
        # self.LrScheduler=lrScheduler

    def forward(self,x, *args: Any, **kwargs: Any) -> Any:
        x=self.Flatten(x)
        y=self.Net(x)
        return y#torch.rand([1,10])
    def configure_optimizers(self) -> Any:
        return [self.Optimizer]
    def training_step(self, batch,*args: Any, **kwargs: Any) -> STEP_OUTPUT:
        x,y=batch
        print(x.shape)
        y_hat=self.forward(x)
        loss=self.LossFunction(y_hat,y)
        acc=accuracy(y_hat,y,task='multiclass',num_classes=10)
        self.log("train_acc", acc, on_step=True, on_epoch=True)
        self.log("trainloss", loss, on_step=True, on_epoch=True)
        return {'loss':loss}
    def validation_step(self,batch, *args: Any, **kwargs: Any) -> STEP_OUTPUT | None:
        x,y=batch
        # x=self.Flatten(x)
        y_hat=self.Net(x)
        loss=self.LossFunction(y_hat,y)
        acc=accuracy(y_hat,y,task='multiclass',num_classes=10)
        self.log("test_acc", acc, on_step=False, on_epoch=True)
        self.log("testloss", loss, on_step=False, on_epoch=True)
        return {'loss':loss}
        
    


In [60]:
from torchsummary import summary
from project.model.MLP import *
net=MLP(28*28,[1024,256],10)
plnet=PLMLP(net,lossFuncthion='none',optimizer=torch.optim.Adam(net.parameters(),lr=0.01),lrScheduler=None)
summary(plnet,(1,28,28),batch_size=2,device='cpu')

torch.Size([2, 1, 28, 28])
torch.Size([2, 784])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Flatten-1                   [2, 784]               0
            Linear-2                  [2, 1024]         803,840
              ReLU-3                  [2, 1024]               0
            Linear-4                   [2, 256]         262,400
              ReLU-5                   [2, 256]               0
            Linear-6                    [2, 10]           2,570
Total params: 1,068,810
Trainable params: 1,068,810
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.05
Params size (MB): 4.08
Estimated Total Size (MB): 4.13
----------------------------------------------------------------


In [61]:

from torch.utils.data import random_split
train, val = random_split(MNISTDataset, [55000, 5000])
trainer = pl.Trainer()
trainer.fit(plnet, DataLoader(train,batch_size=64))#, DataLoader(val,batch_size=64)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
g:\install\anaconda\envs\nni\lib\site-packages\pytorch_lightning\trainer\configuration_validator.py:70: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type             | Params
--------------------------------------------------
0 | Flatten      | Flatten          | 0     
1 | Net          | Sequential       | 1.1 M 
2 | LossFunction | CrossEntropyLoss | 0     
--------------------------------------------------
1.1 M     Trainable params
0         Non-trainable params
1.1 M     Total params
4.275     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 784])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 784])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 784])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 784])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 784])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 784])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 784])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 784])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 784])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 784])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 784])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 784])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Si

In [86]:
net=MLP(28*28,[1024,256],10)
out=flatten(MNISTDataset[0][0])
net(out)

tensor([[ 0.0204, -0.0283,  0.0571, -0.0010, -0.0645, -0.0911, -0.0292,  0.1345,
          0.0189,  0.0499]], grad_fn=<AddmmBackward0>)

In [82]:
from torchsummary import summary
summary(plnet,(1,28,28),batch_size=2,device='cpu')

torch.Size([2, 784])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Flatten-1                   [2, 784]               0
            Linear-2                  [2, 1024]         803,840
              ReLU-3                  [2, 1024]               0
            Linear-4                   [2, 256]         262,400
              ReLU-5                   [2, 256]               0
            Linear-6                    [2, 10]           2,570
Total params: 1,068,810
Trainable params: 1,068,810
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.05
Params size (MB): 4.08
Estimated Total Size (MB): 4.13
----------------------------------------------------------------


In [83]:
from torch.utils.data import DataLoader
batch=next(iter(DataLoader(train,batch_size=3)))

In [79]:
len(batch)
batch[0].shape

torch.Size([3, 1, 28, 28])

In [7]:
from torch.utils.data import random_split
train, val = random_split(MNISTDataset, [55000, 5000])
trainer = pl.Trainer(accelerator="auto")
trainer.fit(plnet, DataLoader(train), DataLoader(val))

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


g:\install\anaconda\envs\nni\lib\site-packages\pytorch_lightning\loops\utilities.py:70: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type             | Params
--------------------------------------------------
0 | Flatten      | Flatten          | 0     
1 | Net          | Sequential       | 1.1 M 
2 | LossFunction | CrossEntropyLoss | 0     
--------------------------------------------------
1.1 M     Trainable params
0         Non-trainable params
1.1 M     Total params
4.275     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

g:\install\anaconda\envs\nni\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:432: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
g:\install\anaconda\envs\nni\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:432: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

torch.Size([1, 1, 28, 28])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (28x28 and 784x1024)